In [ ]:
#In this notebook we scrape vegas odds for upcoming fights and save the data to vegas_odds.json for use on website

In [1]:
#needed after restructuring of directory
import os
os.chdir('/Users/alexchandler/UFC_Prediction_2022')

#!/usr/bin/env python
# coding: utf-8

# In[ ]:

import requests
import urllib3
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import re
from datetime import datetime, timedelta
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)

#turning csv files to json files
import csv
import json
 
# Function to convert a CSV to JSON
# Takes the file paths as arguments
def make_json(csvFilePath, jsonFilePath, column):
     
    # create a dictionary
    data = {}
     
    # Open a csv reader called DictReader
    with open(csvFilePath, encoding='utf-8') as csvf:
        csvReader = csv.DictReader(csvf)
         
        # Convert each row into a dictionary
        # and add it to data
        for rows in csvReader:
             
            # primary key given by column variable
            key = rows[column]
            data[key] = rows
 
    # Open a json writer, and use the json.dumps()
    # function to dump data
    with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
        jsonf.write(json.dumps(data, indent=4))
        
def get_odds_two_rows_per_fight():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    return odds_df


#problem: the fights in the "future events" category do not get lined up properly
def get_odds():
    url = 'https://www.bestfightodds.com'
    page = requests.get(url)
    soup = BeautifulSoup(page.content, "html.parser") 
    mydivs = soup.find_all("tr", {"class": ""})
    rows=[tr for tr in mydivs if 'bestbet' in str(tr)]
    names=[]
    oddsDicts=[]
    books=['DraftKings','BetMGM','Caesars','BetRivers','FanDuel','PointsBet','Unibet','Bet365','BetWay','5D','Ref']
    for row in rows:
        #gets name of fighter in row
        name = row.find_all("span", {"class": "t-b-fcc"})[0].text
        oddsList=[name]
        i=0
        for stat in row.select('td'):
            i+=1
            if i>11:
                break
            try:
                odds = stat.select('span')[0].text
                oddsList.append(odds)
            except:
                oddsList.append('')
        if name in names:
            names.append(name+'.')
        else:
            names.append(name)
        oddsDicts.append(dict(zip(['name']+books,oddsList)))
    oddsDict = dict(zip(names,oddsDicts))
    names = list(oddsDict.keys())
    row0=oddsDict[list(oddsDict.keys())[0]]
    odds_df = pd.DataFrame(row0, index=[0])
    for i in range(1,len(names)):
        row=oddsDict[names[i]]
        odds_df = pd.concat([odds_df, pd.DataFrame(row, index=[i])], axis = 0)
    #making it so each fight has just a single row instead of two rows
    #making dataframe just for even indexed columns
    odds_df_evens = odds_df[odds_df.index%2==0]
    newcolumns1={}
    for col in list(odds_df_evens.columns):
        newcolumns1[col]='fighter '+col
    odds_df_evens=odds_df_evens.rename(columns=newcolumns1)
    odds_df_evens.reset_index(drop=True, inplace=True)
    #making dataframe just for odd indexed columns
    odds_df_odds = odds_df[odds_df.index%2==1]
    newcolumns2={}
    for col in list(odds_df_odds.columns):
        newcolumns2[col]='opponent '+col
    odds_df_odds=odds_df_odds.rename(columns=newcolumns2)
    odds_df_odds.reset_index(drop=True, inplace=True)
    new_odds_df = pd.concat([odds_df_evens, odds_df_odds], axis = 1)
    return new_odds_df


In [2]:
os.chdir('/Users/alexchandler/UFC_Prediction_2022/src')
from functions import *
os.chdir('/Users/alexchandler/UFC_Prediction_2022')

In [3]:
ufc_fights

,date,division,fighter,opponent,result,method,fighter_wins,fighter_losses,fighter_age,fighter_height,fighter_reach,fighter_L5Y_wins,fighter_L5Y_losses,fighter_L2Y_wins,fighter_L2Y_losses,fighter_ko_wins,fighter_ko_losses,fighter_L5Y_ko_wins,fighter_L5Y_ko_losses,fighter_L2Y_ko_wins,fighter_L2Y_ko_losses,fighter_sub_wins,fighter_sub_losses,fighter_L5Y_sub_wins,fighter_L5Y_sub_losses,fighter_L2Y_sub_wins,fighter_L2Y_sub_losses,fighter_inf_knockdowns_avg,fighter_inf_pass_avg,fighter_inf_reversals_avg,fighter_inf_sub_attempts_avg,fighter_inf_takedowns_landed_avg,fighter_inf_takedowns_attempts_avg,fighter_inf_sig_strikes_landed_avg,fighter_inf_sig_strikes_attempts_avg,fighter_inf_total_strikes_landed_avg,fighter_inf_total_strikes_attempts_avg,fighter_inf_head_strikes_landed_avg,fighter_inf_head_strikes_attempts_avg,fighter_inf_body_strikes_landed_avg,fighter_inf_body_strikes_attempts_avg,fighter_inf_leg_strikes_landed_avg,fighter_inf_leg_strikes_attempts_avg,fighter_inf_distance_strikes_landed_avg,fighter_inf_distance_strikes_attempts_avg,fighter_inf_clinch_strikes_landed_avg,fighter_inf_clinch_strikes_attempts_avg,fighter_inf_ground_strikes_landed_avg,fighter_inf_ground_strikes_attempts_avg,fighter_abs_knockdowns_avg,fighter_abs_pass_avg,fighter_abs_reversals_avg,fighter_abs_sub_attempts_avg,fighter_abs_takedowns_landed_avg,fighter_abs_takedowns_attempts_avg,fighter_abs_sig_strikes_landed_avg,fighter_abs_sig_strikes_attempts_avg,fighter_abs_total_strikes_landed_avg,fighter_abs_total_strikes_attempts_avg,fighter_abs_head_strikes_landed_avg,fighter_abs_head_strikes_attempts_avg,fighter_abs_body_strikes_landed_avg,fighter_abs_body_strikes_attempts_avg,fighter_abs_leg_strikes_landed_avg,fighter_abs_leg_strikes_attempts_avg,fighter_abs_distance_strikes_landed_avg,fighter_abs_distance_strikes_attempts_avg,fighter_abs_clinch_strikes_landed_avg,fighter_abs_clinch_strikes_attempts_avg,fighter_abs_ground_strikes_landed_avg,fighter_abs_ground_strikes_attempts_avg,opponent_wins,opponent_losses,opponent_age,opponent_height,opponent_reach,opponent_L5Y_wins,opponent_L5Y_losses,opponent_L2Y_wins,opponent_L2Y_losses,opponent_ko_wins,opponent_ko_losses,opponent_L5Y_ko_wins,opponent_L5Y_ko_losses,opponent_L2Y_ko_wins,opponent_L2Y_ko_losses,opponent_sub_wins,opponent_sub_losses,opponent_L5Y_sub_wins,opponent_L5Y_sub_losses,opponent_L2Y_sub_wins,opponent_L2Y_sub_losses,opponent_inf_knockdowns_avg,opponent_inf_pass_avg,opponent_inf_reversals_avg,opponent_inf_sub_attempts_avg,opponent_inf_takedowns_landed_avg,opponent_inf_takedowns_attempts_avg,opponent_inf_sig_strikes_landed_avg,opponent_inf_sig_strikes_attempts_avg,opponent_inf_total_strikes_landed_avg,opponent_inf_total_strikes_attempts_avg,opponent_inf_head_strikes_landed_avg,opponent_inf_head_strikes_attempts_avg,opponent_inf_body_strikes_landed_avg,opponent_inf_body_strikes_attempts_avg,opponent_inf_leg_strikes_landed_avg,opponent_inf_leg_strikes_attempts_avg,opponent_inf_distance_strikes_landed_avg,opponent_inf_distance_strikes_attempts_avg,opponent_inf_clinch_strikes_landed_avg,opponent_inf_clinch_strikes_attempts_avg,opponent_inf_ground_strikes_landed_avg,opponent_inf_ground_strikes_attempts_avg,opponent_abs_knockdowns_avg,opponent_abs_pass_avg,opponent_abs_reversals_avg,opponent_abs_sub_attempts_avg,opponent_abs_takedowns_landed_avg,opponent_abs_takedowns_attempts_avg,opponent_abs_sig_strikes_landed_avg,opponent_abs_sig_strikes_attempts_avg,opponent_abs_total_strikes_landed_avg,opponent_abs_total_strikes_attempts_avg,opponent_abs_head_strikes_landed_avg,opponent_abs_head_strikes_attempts_avg,opponent_abs_body_strikes_landed_avg,opponent_abs_body_strikes_attempts_avg,opponent_abs_leg_strikes_landed_avg,opponent_abs_leg_strikes_attempts_avg,opponent_abs_distance_strikes_landed_avg,opponent_abs_distance_strikes_attempts_avg,opponent_abs_clinch_strikes_landed_avg,opponent_abs_clinch_strikes_attempts_avg,opponent_abs_ground_strikes_landed_avg,opponent_abs_ground_strikes_attempts_avg,fighter_stance,

In [4]:
same_name('alekander rakic', 'aleksandr rakic')

True

In [4]:
#scrapes odds from bestfightodds.com
odds_df = get_odds();odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
0,Chris Vasil,,,,,,,,,-120,-130,-120,Vilson Ndregjoni,,,,,,,,,-110,+100,-110
1,David Meshkhoradze,,,,,,,,,+115,+130,+125,Thomas Majeski,,,,,,,,,-150,-160,-155
2,Efren Escareno,,,,,,,,,+300,+400,+400,Teshay Gouthro,,,,,,,,,-450,-550,-550
3,David Ciuclea,,,,,,,,,+300,+420,+425,Ian Allston,,,,,,,,,-450,-590,-600
4,Manoel Sousa,,,,,,,,,-1000,-1475,-1300,Mauro Chaulet,,,,,,,,,+500,+800,+900
5,Feraris Golden Jr,,,,,,,,,-163,-160,-155,Kendly St. Louis,,,,,,,,,+120,+130,+125
6,Joel Kouadja,+550,+500,+475,,,,,+475,+450,+500,+500,Matthew Bonner,-800,-750,-650,,,,,-714,-699,-800,-800
7,Bogdan Barbu,+475,+500,+550,,,,,+525,+450,+475,+500,Daan Duijs,-675,-750,-800,,,,,-833,-800,-735,-700
8,Adam Cullen,-900,-1200,-1000,,,,,-667,-800,-1100,-1500,Arnaud Kherfallah,+600,+700,+650,,,,,+450,+450,+620,+900
9,Benoit Blanc,+170,+155,+160,,,,,+165,+150,+150,+155,Tanio Pagliariccio,-200,-190,-190,,,,,-200,-200,-180,-185


In [5]:
#thresh is the number of bookies we allow to not have odds on the books
def drop_irrelevant_fights(df,thresh):
    irr=[]
    for i in df.index:
        count=0
        row=list(df.loc[i])
        for j in row:
            if j=='':
                count+=1
        if count>2*thresh:
            irr.append(i)
    df=df.drop(irr)
    return df

#thresh is the number of bookies we allow to not have odds on the books
def drop_non_ufc_fights(df):
    irr=[]
    for i in df.index:
        if (not in_ufc(odds_df['fighter name'][i])) or (not in_ufc(odds_df['opponent name'][i])):
            irr.append(i)
    df=df.drop(irr)
    return df

#thresh is the number of bookies we allow to not have odds on the books
def drop_repeats(df):
    irr=[]
    for i in df.index:
        fname=odds_df['fighter name'][i]
        oname=odds_df['opponent name'][i]
        for j in range(100):
            fname_old=ufc_fights['fighter'][j]
            oname_old=ufc_fights['opponent'][j]
            if (same_name(fname, fname_old) and same_name(oname, oname_old)) or (same_name(oname, fname_old) and same_name(fname, oname_old)):
                irr.append(i)
    df=df.drop(irr)
    return df

In [6]:
odds_df=drop_irrelevant_fights(odds_df,1)

In [7]:
odds_df=drop_non_ufc_fights(odds_df)

In [8]:
odds_df=drop_repeats(odds_df)

In [9]:
odds_df

,fighter name,fighter DraftKings,fighter BetMGM,fighter Caesars,fighter BetRivers,fighter FanDuel,fighter PointsBet,fighter Unibet,fighter Bet365,fighter BetWay,fighter 5D,fighter Ref,opponent name,opponent DraftKings,opponent BetMGM,opponent Caesars,opponent BetRivers,opponent FanDuel,opponent PointsBet,opponent Unibet,opponent Bet365,opponent BetWay,opponent 5D,opponent Ref
30,Alexander Volkov,-150,-165,-160,-175,-156,,-175,-152,-163,-155,-153,Jairzinho Rozenstruik,+130,+135,+135,+133,+132,,+133,+125,+130,+135,+133
34,Benoit Saint-Denis,-160,-155,-150,-159,-162,,-159,-161,-150,-155,-155,Niklas Stolze,+140,+130,+125,+125,+136,,+125,+130,+110,+135,+135
35,Alex da Silva,+150,+140,+150,+145,+146,,+145,+150,+150,+145,+145,Joe Solecki,-170,-165,-175,-186,-174,,-186,-175,-188,-165,-170
37,Dan Ige,+290,+300,+310,+270,+300,,+270,+275,+275,+305,+300,Movsar Evloev,-380,-400,-400,-375,-375,,-375,-357,-400,-365,-400
38,Jeff Molina,-190,-190,-190,-186,-190,,-186,-189,-188,-180,-180,Zhalgas Zhumagulov,+160,+155,+160,+145,+160,,+145,+162,+150,+158,+155
39,Ode' Osbourne,-170,-175,-175,-180,-174,,-180,-172,-163,-170,-173,Zarrukh Adashev,+150,+145,+150,+140,+146,,+140,+140,+120,+150,+148
40,Erin Blanchfield,-550,-600,-575,-625,-620,,-625,-714,-549,-600,-600,Jj Aldrich,+400,+425,+425,+410,+460,,+410,+500,+350,+450,+400
43,Glover Teixeira,+165,+165,+170,+165,+160,+160,+165,+162,+163,+165,+165,Jiri Prochazka,-195,-200,-200,-210,-190,-195,-210,-189,-200,-190,-190
44,Taila Santos,+500,+500,+550,+510,+530,+550,+510,+550,+500,+525,+425,Valentina Shevchenko,-720,-750,-800,-835,-750,-800,-835,-833,-800,-750,-600
45,Joanna Jedrzejczyk,+150,+145,+150,+153,+152,,+153,+150,+140,+158,+150,Zhang Weili,-170,-175,-175,-195,-180,,-195,-175,-188,-180,-175


In [ ]:
good_indices = [24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34]

In [ ]:
odds_df=odds_df.loc[good_indices];odds_df

In [ ]:
n=77
row = odds_df.loc[n].to_frame()
df = odds_df.drop([n])
row.index
new_row = pd.DataFrame(dict(zip(row.index,list(row[n]))), index=[0])

In [ ]:
new_row

In [ ]:
odds_df = pd.concat([new_row, df]).reset_index(drop = True)

In [ ]:
odds_df

In [ ]:
#saves the scraped odds to the file vegas_odds.json in src/models/buildingMLModel/data/external/
result = odds_df.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
#run this if you want to see what the dataframe looks like before row doubling
odds_df_two=get_odds_two_rows_per_fight();odds_df_two

In [ ]:
#use this if you need to add custom rows to vegas_odds

In [ ]:
# if you want to add a new row from bestfightodds.com, copy and paste the two rows, 
# find and replace the arrows with commas, delete extra commas, turn names into strings, 
new_row_data = ['Charles Oliveira',-170,-165,-170,-177,-172,-165,-177,-164,-175,-170,-170,
'Justin Gaethje',+150,+140,+145,+140,+140,+136,+140,+137,+140,+150,+145]

#make data into dataframe column with correct column names and add to odds_df
new_row_data = dict(zip(odds_df.columns,new_row_data))
new_row = pd.DataFrame(new_row_data, index =[0])
odds_df = pd.concat([new_row, odds_df]).reset_index(drop = True);odds_df

In [ ]:
#if it looks ok, save it
#IMPORTANT. NEXT GO TO UFC_Prediction_Model notebook and run predictions after exporting vegas_odds.json
#saves the scraped odds to the file vegas_odds.json in src/models/buildingMLModel/data/external/
result = odds_df.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/vegas_odds.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
# BELOW WE MANUALLY FIX PREDICTION HISTORY IF IT EVER GETS MESSED UP
# IF YOU DON'T NEED TO FIX PREDICTION_HISTORY, DO NOT RUN THE CELLS BELOW

In [ ]:
#sometimes prediction_history gets messed up... here we can fix it manually
prediction_history = pd.read_json('src/models/buildingMLModel/data/external/prediction_history.json');prediction_history

In [ ]:
#to drop rows
prediction_history=prediction_history.drop([0,1,2,3,4]);prediction_history

In [ ]:
#if it looks ok, save it
#IMPORTANT. NEXT GO TO UFC_Prediction_Model notebook and run predictions after exporting vegas_odds.json
#saves the scraped odds to the file vegas_odds.json in src/models/buildingMLModel/data/external/
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
#make dataframe with vig added for bets with James
def add_vig(odds):
    return 0
predictions_with_vig = prediction_history.copy()
predictions_with_vig=predictions_with_vig.iloc[0:9][['fighter name', 'opponent name','predicted fighter odds','predicted opponent odds']]



In [ ]:
type(predictions_with_vig['predicted fighter odds'][0])

In [ ]:
#found some rows that should not be there and removing by hand (need to fix python scripts to avoid having to do this)
for i in [21,23]:
    prediction_history=prediction_history.drop([i])
prediction_history

In [ ]:
result = prediction_history.to_json()
parsed = json.loads(result)
jsonFilePath='src/models/buildingMLModel/data/external/prediction_history.json'
with open(jsonFilePath, 'w', encoding='utf-8') as jsonf:
    jsonf.write(json.dumps(parsed, indent=4))
print('saved to '+jsonFilePath)

In [ ]:
prediction_history

In [ ]:
#use this to fix any names in the dataframe
prediction_history.at[7, 'opponent name'] = 'Marcos Rogerio de lima'
prediction_history